In [3]:
""" !pip install pytube
!pip install openai-whisper
!pip install langchain
!pip install openai
!pip install python-dotenv """

' !pip install pytube\n!pip install openai-whisper\n!pip install langchain\n!pip install openai\n!pip install python-dotenv '

In [4]:
""" !pip install -U langchain-community
!pip install pinecone-client
!pip install --upgrade pytube
!pip install yt-dlp
!pip install imageio[ffmpeg] """

' !pip install -U langchain-community\n!pip install pinecone-client\n!pip install --upgrade pytube\n!pip install yt-dlp\n!pip install imageio[ffmpeg] '

In [5]:
from torch import cuda, torch
device = f'cuda:{cuda.current_device()}' if cuda.is_available() else 'cpu'

In [6]:
from dotenv import load_dotenv, find_dotenv
import getpass
import os

# Load Environment Variable
_ = load_dotenv(find_dotenv())

# Set the environment variable directly in the script
os.environ["OPENAI_API_KEY"] = getpass.getpass("Enter OpenAI API Key: ")

In [9]:
#print(os.environ["OPENAI_API_KEY"])

In [11]:
#!pip install langsmith

In [12]:
# Import Langsmith (make sure it's installed first)
import langsmith  # Importing Langsmith's library
from langsmith import traceable
from langsmith.wrappers import wrap_openai
from langsmith import Client
from langsmith.evaluation import evaluate
from langsmith.schemas import Example, Run
import getpass
import os

In [13]:
os.environ["LANGCHAIN_TRACING_V2"] = "true"
os.environ["LANGCHAIN_ENDPOINT"] = "https://api.smith.langchain.com"
os.environ["LANGCHAIN_PROJECT"] = "prj-yt-chatbot-qa"
os.environ["LANGCHAIN_API_KEY"] = getpass.getpass("Enter LANGCHAIN_API_KEY")        # Authenticate with Langsmith

In [15]:
#print(os.environ["LANGCHAIN_API_KEY"])

In [16]:
# STEP 1: Load VIDEO, Download Audio, Transcribe into different text files!!!

import os
import yt_dlp
from whisper import load_model
from langchain.agents import initialize_agent, Tool
from langchain.schema import HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI

# Default configurations
VIDEO_LINKS_FILE = "video_links.txt"
AUDIO_OUTPUT_TEMPLATE = "audio_{}"
WHISPER_MODEL = "base"
TRANSCRIPT_DIR = "transcriptions"

# Ensure directory exists for transcriptions
os.makedirs(TRANSCRIPT_DIR, exist_ok=True)

# Load YouTube links from the file
def load_video_links(filename=VIDEO_LINKS_FILE):
    with open(filename, 'r') as file:
        return [link.strip() for link in file.readlines() if link.strip()]

# Download audio from YouTube video
def download_audio(url, output_name):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [{'key': 'FFmpegExtractAudio', 'preferredcodec': 'mp3', 'preferredquality': '192'}],
        'outtmpl': f"{output_name}",
        'ffmpeg_location': '/usr/bin/ffmpeg'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return f"{output_name}.mp3"

# Transcribe audio using Whisper
def transcribe_audio_with_whisper(audio_path, model_name=WHISPER_MODEL):
    model = load_model(model_name)
    result = model.transcribe(audio_path)
    return result["text"]

# Process each video and save individual transcriptions
def transcribe_all_videos():
    video_links = load_video_links(VIDEO_LINKS_FILE)

    for i, video_url in enumerate(video_links):
        print(f"Processing video {i+1}/{len(video_links)}: {video_url}")
        audio_path = download_audio(video_url, AUDIO_OUTPUT_TEMPLATE.format(i))
        
        # Transcribe the audio and save each transcription to a separate file
        transcription = transcribe_audio_with_whisper(audio_path)
        transcript_file = os.path.join(TRANSCRIPT_DIR, f"transcription_{i+1}.txt")
        
        with open(transcript_file, 'w') as f:
            f.write(transcription)
        
        print(f"Completed transcription for video {i+1}")

# Execute transcription process only, without consolidation
if __name__ == "__main__":
    transcribe_all_videos()
    print("Individual transcription files are ready in the transcriptions folder.")


Processing video 1/8: https://www.youtube.com/watch?v=-RIfno4Sp0g
[youtube] Extracting URL: https://www.youtube.com/watch?v=-RIfno4Sp0g
[youtube] -RIfno4Sp0g: Downloading webpage
[youtube] -RIfno4Sp0g: Downloading ios player API JSON
[youtube] -RIfno4Sp0g: Downloading mweb player API JSON
[youtube] -RIfno4Sp0g: Downloading player 81ef9024
[youtube] -RIfno4Sp0g: Downloading m3u8 information
[info] -RIfno4Sp0g: Downloading 1 format(s): 251
[download] Destination: audio_0
[download] 100% of   48.70MiB in 00:00:00 at 51.13MiB/s    
[ExtractAudio] Destination: audio_0.mp3
Deleting original file audio_0 (pass -k to keep)


100%|███████████████████████████████████████| 139M/139M [00:02<00:00, 57.6MiB/s]


Completed transcription for video 1
Processing video 2/8: https://www.youtube.com/watch?v=3SCOVB1KDFc
[youtube] Extracting URL: https://www.youtube.com/watch?v=3SCOVB1KDFc
[youtube] 3SCOVB1KDFc: Downloading webpage
[youtube] 3SCOVB1KDFc: Downloading ios player API JSON
[youtube] 3SCOVB1KDFc: Downloading mweb player API JSON
[youtube] 3SCOVB1KDFc: Downloading m3u8 information
[info] 3SCOVB1KDFc: Downloading 1 format(s): 251
[download] Destination: audio_1
[download] 100% of   54.16MiB in 00:00:02 at 18.42MiB/s    
[ExtractAudio] Destination: audio_1.mp3
Deleting original file audio_1 (pass -k to keep)
Completed transcription for video 2
Processing video 3/8: https://www.youtube.com/watch?v=qMqP2IwhSnA
[youtube] Extracting URL: https://www.youtube.com/watch?v=qMqP2IwhSnA
[youtube] qMqP2IwhSnA: Downloading webpage
[youtube] qMqP2IwhSnA: Downloading ios player API JSON
[youtube] qMqP2IwhSnA: Downloading mweb player API JSON
[youtube] qMqP2IwhSnA: Downloading m3u8 information
[info] qMqP2Iw

In [17]:
# STEP 2: Define Pinecone as Vector DB & Define Index

from pinecone import Pinecone
import os
from getpass import getpass

# Securely fetch Pinecone API keys                    
os.environ["PINECONE_API_KEY"] = os.getenv("PINECONE_API_KEY") or getpass("Enter Pinecone API key: ")

In [22]:
#print(os.environ["PINECONE_API_KEY"])
from pinecone import ServerlessSpec

spec = ServerlessSpec(
    cloud="aws", region="us-east-1"
)

In [24]:
# Create INDEX 

import os
import time
from tqdm.auto import tqdm
from pinecone import Pinecone, ServerlessSpec

# Initialize Pinecone client with the updated API
pc = Pinecone(api_key=os.environ["PINECONE_API_KEY"])
#spec = ServerlessSpec(cloud="aws", region="us-west-1")          # Specify Pinecone environment
index_name = "transcription-accnt-earn-index-no-consol"             # Create New Index

# Check if the index exists; create if it doesn’t
if index_name not in [index_info["name"] for index_info in pc.list_indexes()]:
    pc.create_index(
        name=index_name,
        dimension=1536,  # OpenAI embedding dimension
        metric="dotproduct",
        spec=spec
    )
    # Wait for the index to be ready
    while not pc.describe_index(index_name).status["ready"]:
        print("Waiting for index to be ready...")
        time.sleep(1)

# Connect to the index for upsertion
index = pc.Index(index_name)
time.sleep(1)
# view index stats
print("Index statistics:", index.describe_index_stats())

Index statistics: {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}


In [28]:
# STEP 3: Vectorize and Embed each Transcription file

from langchain.embeddings import OpenAIEmbeddings
from langchain.text_splitter import RecursiveCharacterTextSplitter
import os

# Initialize embedding model
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002")

# Function to split and vectorize text from each transcription file
def vectorize_individual_transcriptions(transcription_dir="transcriptions"):
    # Initialize list to hold vectorized segments for all files
    all_vectorized_segments = []
    
    # Iterate through each transcription file in the directory
    for filename in sorted(os.listdir(transcription_dir)):
        file_path = os.path.join(transcription_dir, filename)
        
        # Skip directories and hidden files
        if not os.path.isfile(file_path) or filename.startswith('.'):
            continue

        # Read transcription text
        with open(file_path, 'r') as f:
            transcription_text = f.read()
        
        # Split text into chunks
        text_splitter = RecursiveCharacterTextSplitter(chunk_size=1500, chunk_overlap=200)
        chunks = text_splitter.split_text(transcription_text)
        
        # Vectorize each chunk with metadata
        for chunk_idx, chunk in enumerate(chunks):
            embedding = embedding_model.embed_documents([chunk])[0]  # Get embedding
            
            # Metadata includes file identifier and chunk index for reference
            metadata = {
                "file_name": filename,
                "file_id": filename.replace("transcription_", "").replace(".txt", ""),
                "chunk_idx": chunk_idx,
            }
            all_vectorized_segments.append({
                "embedding": embedding,
                "text": chunk,
                "metadata": metadata
            })
        
        print(f"Vectorization complete for {filename}.")

    return all_vectorized_segments

# Execute the vectorization task
if __name__ == "__main__":
    vectorized_segments = vectorize_individual_transcriptions()
    print("All transcription files have been vectorized. Ready for storage in Pinecone.")


Vectorization complete for transcription_1.txt.
Vectorization complete for transcription_2.txt.
Vectorization complete for transcription_3.txt.
Vectorization complete for transcription_4.txt.
Vectorization complete for transcription_5.txt.
Vectorization complete for transcription_6.txt.
Vectorization complete for transcription_7.txt.
Vectorization complete for transcription_8.txt.
All transcription files have been vectorized. Ready for storage in Pinecone.


In [47]:
# STEP 4 - UPSERT embeddings to Pinecone

from tqdm.auto import tqdm

# Function to upsert vectorized segments to Pinecone
def upsert_embeddings_to_pinecone(vectorized_segments, index):
    for segment in tqdm(vectorized_segments, desc="Upserting segments to Pinecone"):
        # Prepare data for upsert
        segment_id = segment["metadata"]["file_id"] + "_" + str(segment["metadata"]["chunk_idx"])  # Unique ID for each chunk
        embedding = segment["embedding"]
        metadata = segment["metadata"]
        metadata["text"] = segment["text"]  # Adds the chunk's text content

        # Upsert the segment into Pinecone
        try:
            index.upsert(vectors=[(segment_id, embedding, metadata)])
        except Exception as e:
            print(f"Error upserting segment {segment_id}: {e}")

    print("All segments upserted into Pinecone.")

# Example usage:
upsert_embeddings_to_pinecone(vectorized_segments, index)


Upserting segments to Pinecone:   0%|          | 0/349 [00:00<?, ?it/s]

All segments upserted into Pinecone.


In [48]:
print("Index statistics:", index.describe_index_stats())

Index statistics: {'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {'': {'vector_count': 349}},
 'total_vector_count': 349}


In [ ]:
#!pip install langchain-community
#!pip install google-search-results

In [ ]:
## !pip install tavily-python (not needed)

In [38]:
import os
import getpass

#os.environ["TAVILY_API_KEY"] = os.getenv("TAVILY_API_KEY") or getpass.getpass("Enter Tavily API key: ")

# Fetch the SerpAPI key securely
os.environ["SERPAPI_API_KEY"] = os.getenv("SERPAPI_API_KEY") or getpass.getpass("Enter SerpAPI key: ")

# Example usage in SerpAPI tool setup
serp_api_key = os.environ["SERPAPI_API_KEY"]

In [40]:
#print(os.environ["SERPAPI_API_KEY"])

In [50]:
# DEFINE ROUTER, RETRIEVE from VectorstoreDB and Retrieve from Google search functions!

import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, SystemMessage, AIMessage
from langchain.vectorstores import Pinecone
from langchain.embeddings import OpenAIEmbeddings
from serpapi import GoogleSearch
import os

# Initialize the embedding model and vector store
embedding_model = OpenAIEmbeddings(model="text-embedding-ada-002", openai_api_key=os.environ["OPENAI_API_KEY"])
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])

# Pinecone index is already set up and embeddings are upserted
vector_store = Pinecone(index=index, embedding=embedding_model, text_key="text")

# Initialize the router instructions
router_instructions = """You are an expert at routing a user question to a vectorstore or web search.

The vectorstore contains documents related to Accenture company Quarterly results, Revenue, Earnings for the last four quarters, sales, digital projects, cloud projects and employee details.

Use the vectorstore for questions on these topics. For all else, and especially for current events, use web-search.

Return JSON with a single key, "datasource", that is either 'websearch' or 'vectorstore' depending on the question."""

# Function to route query based on instructions
def route_query(query):
    routing_response = chat_model.invoke(
        [SystemMessage(content=router_instructions), HumanMessage(content=query)]
    )
    route_decision = json.loads(routing_response.content)
    return route_decision["datasource"]

# Function to retrieve from vector database
def retrieve_from_vectorstore(query):
    retriever = vector_store.as_retriever()
    results = retriever.get_relevant_documents(query)
    document_text = "\n\n".join([doc.page_content for doc in results])

    # Summarize the retrieved documents using LLM 
    prompt = (
        "Based on the following documents, provide a concise answer to the question:\n\n"
        f"Question: {query}\n\n"
        "Documents:\n"
        f"{document_text}\n\n"
        "Answer:"
    )    
    answer_response = chat_model.invoke([HumanMessage(content=prompt)])
    return answer_response.content
    
# Function to retrieve from Google search
def web_search(query):
    search = GoogleSearch({
        "q": query,
        "api_key": serp_api_key, 
        "num": 3  # Limit to top 3 results for conciseness
    })
    results = search.get_dict()

    # Extract relevant text from search results
    combined_text = "\n".join(
        result.get("snippet", "No snippet available")
        for result in results.get("organic_results", [])
    )
    
    # Summarize results using the LLM
    prompt = (
        f"Summarize the following search results to answer the question:\n\n"
        f"Question: {query}\n\n"
        f"Results:\n{combined_text}\n\n"
        "Answer:"
    )
    summary_response = chat_model.invoke([HumanMessage(content=prompt)])
    return summary_response.content  

# Main function to handle the query routing and retrieval
def handle_query(query):
    datasource = route_query(query)
    
    if datasource == "vectorstore":
        print("Routing to Vectorstore...")
        return retrieve_from_vectorstore(query)
    elif datasource == "websearch":
        print("Routing to Web Search...")
        return web_search(query)
    else:
        return "Unknown data source. Please check the router configuration."

In [ ]:
# DEFINE ALL THE TOOLS for ReACT agent! 

from langchain.agents import initialize_agent
from langchain.agents import Tool
from langchain.chat_models import ChatOpenAI
from serpapi import GoogleSearch
from langchain.memory import ConversationBufferMemory
import os

# Initialize memory
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)


# Define the Web search tool using SerpAPI
web_search_tool = Tool(
    name="Google Search",
    func=web_search,
    description="Retrieve relevant information using google web search for general or current event questions."
)

# Define tool for vector store retrieval
vector_store_tool = Tool(
    name="VectorStore Retrieval",
    func=retrieve_from_vectorstore,
    description="Retrieve relevant documents from the vector store for questions about Accenture's earnings, revenue, sales, projects, and employee details.",
)

# Revised prompt for the agent with improved answer formatting
agent_prompt = """
You are an expert assistant specializing in Accenture's financial data and competitor analysis.

For each user query:
1. Answer strictly based on provided context or vector data.
2. Be concise and use up to three sentences.
3. If the answer is found, respond directly by starting with "Final Answer:" without additional steps or thoughts.

Now, respond to the question:
{user_query}
"""


# Define the ReAct agent with the custom prompt in each tool
react_agent = initialize_agent(
    tools=[vector_store_tool, web_search_tool],
    agent="zero-shot-react-description",
    llm=chat_model,
    memory=memory,  # Integrate conversation memory
    verbose=True,
    max_iterations=5  # Restrict maximum depth to avoid infinite loops
    handle_parsing_errors=True  # Retry on output parsing errors
)

# Main function to handle the query using the ReAct agent with the custom prompt
def handle_query_with_agent(query):
    # Format the query with the agent's prompt
    formatted_prompt = agent_prompt.format(user_query=query)
    
    # Use the prompt in the agent invocation
    try:
        response = react_agent({"input": formatted_prompt})
        # If a valid answer is provided, return it
        if "output" in response:
            return response["output"]
        else:
            # Fallback message if no answer found within the max iterations
            return "I don't know. Unable to find the answer within the provided context."
    except Exception as e:
        # In case of unexpected errors, provide a helpful message
        return f"Error encountered: {str(e)}"

In [58]:
# INTERACTIVE CHAT FUNCTION using ReACT Agent - REFINED 3! Works perfect! TUESDAY

import json
from langchain.chat_models import ChatOpenAI
from langchain.schema import HumanMessage, AIMessage
from langchain.memory import ConversationBufferMemory

# Initialize chat model and memory
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Function to clear memory
def reset_memory():
    memory.clear()
    print("Memory has been reset.")

# Interactive chat function with datasource-based source tagging
def chatbot_interaction():
    print("Welcome to the Accenture Q&A Chatbot! Ask any question about Accenture.")
    while True:
        user_query = input("You: ")
        if user_query.lower() in ["exit", "quit", "no"]:
            print("Agent: Thank you for using the Q&A chatbot. Goodbye!")
            break
        
        # Determine the datasource
        datasource = route_query(user_query)
        
        # Tag source based on datasource
        source_label = "VectorDB" if datasource == "vectorstore" else "Google Search" if datasource == "websearch" else "Unknown Source"        
        
        try:
            # Get response from the handle_query_with_agent function instead of calling react_agent directly
            answer = handle_query_with_agent(user_query)
            print(f"\nAgent (using {source_label}):\nQuestion: {user_query}\nAnswer: {answer}")
        
        except Exception as e:
            print(f"Error encountered: {e}")
            print("Agent: Unable to process the request. Please try again.")
        
        print("\nAgent: Do you have more questions?")

# Start the chatbot interaction
chatbot_interaction()

Welcome to the Accenture Q&A Chatbot! Ask any question about Accenture.


> Entering new AgentExecutor chain...
I should retrieve the revenue information for Q3 2024 from the vector store.
Action: VectorStore Retrieval
Action Input: Q3 2024 revenue Accenture
Observation: The projected revenue for Accenture in Q3 2024 is expected to be in the range of 15.85 to 16.45 billion, with an estimated 2 to 5 percent growth in local currency compared to the previous year.
Thought:I have found the revenue information for Q3 2024.
Final Answer: The projected revenue for Accenture in Q3 2024 is expected to be in the range of 15.85 to 16.45 billion.

> Finished chain.

Agent (using VectorDB):
Question: revenue of Q3 2024?
Answer: The projected revenue for Accenture in Q3 2024 is expected to be in the range of 15.85 to 16.45 billion.

Agent: Do you have more questions?
Agent: Thank you for using the Q&A chatbot. Goodbye!


In [56]:
#!pip install gradio

In [ ]:
# GRADIO only with TEXT - REFINED! Works Perfect!

import json
import openai
import gradio as gr

def gradio_chatbot(user_query):
    # Determine the source using the routing function
    datasource = route_query(user_query)
    
    # Tag source based on the datasource returned
    source_label = "VectorDB" if datasource == "vectorstore" else "Google Search" if datasource == "websearch" else "Unknown Source"
    
    try:
        # Get response from the ReAct agent
        response = react_agent({"input": user_query})

        # Check if the response is not empty and parse the response JSON if possible
        if response and "output" in response:
            output_content = response["output"]
            
            # Attempt to parse JSON response if available
            try:
                parsed_response = json.loads(output_content) if isinstance(output_content, str) else output_content
                answer = parsed_response.get("answer", output_content) if isinstance(parsed_response, dict) else output_content
            except json.JSONDecodeError:
                answer = output_content  # Fallback to raw output if JSON parsing fails

            return f"Answer (from {source_label}):\n{answer}"

        else:
            return "Error: Received an empty or unexpected response format from the agent."

    except Exception as e:
        # Capture any other errors
        return f"Error encountered: {str(e)}"

# Create the Gradio Interface
iface = gr.Interface(
    fn=gradio_chatbot,  # Function to call
    inputs="text",      # Input type: text
    outputs="text",     # Output type: text
    title="Accenture Q&A Chatbot",
    description="Ask any question about Accenture and get answers with sourced information."
)

# Launch the Gradio app
iface.launch(share=True)

* Running on local URL:  http://127.0.0.1:7860
* Running on public URL: https://05e3852bb6023b35a3.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
I should use VectorStore Retrieval to find this specific financial information.
Action: VectorStore Retrieval
Action Input: "Accenture EPS fourth quarter 2024"
Observation: The Accenture EPS for the fourth quarter of 2024 is not provided in the documents.
Thought:I should try using Google Search to find this specific financial information.
Action: Google Search
Action Input: "Accenture EPS fourth quarter 2024"
Observation: Accenture reported an adjusted EPS of $2.79 for the fourth quarter of fiscal 2024, which was a 3% increase over the previous quarter.
Thought:I now have the final answer.
Final Answer: The EPS of Accenture in the fourth quarter of 2024 was $2.79.

> Finished chain.


> Entering new AgentExecutor chain...
I should use VectorStore Retrieval to find information about Accenture's earnings.
Action: VectorStore Retrieval
Action Input: "Accenture earnings Q3 2024"
Observation: Accenture reported strong growth in FY25, with full fiscal

In [60]:
# GISKARD EVALUATION BELOW !!!

#!pip install --default-timeout=100 "giskard[llm]"
from giskard.rag import KnowledgeBase, generate_testset, evaluate


2024-11-06 14:46:20.283141: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-11-06 14:46:20.283212: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-06 14:46:20.284410: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-11-06 14:46:20.291897: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-06 14:46:21.516180: W tensorflow/compiler/tf2

In [68]:
import pandas as pd

# Define the sample questions to test with `handle_query`
# Define the list of queries to retrieve answers for
queries = [
    "What is the Earnings of Accenture in fourth quarter of 2024?",
    "What is the EPS of Accenture in fourth quarter of 2024?",
    "Who are the competitors of Accenture?",
    "What is the size of Genai project deals for Accenture in year 2024?",
    "what is the total employee count in Accenture today??"    
]

# Collect answers by running each query through the handle_query function
answers = [handle_query(query) for query in queries]

# Create a DataFrame with only the answers
df_giskard = pd.DataFrame(answers, columns=["text"])

# Display the DataFrame to verify
df_giskard.head()


Routing to Vectorstore...
Routing to Vectorstore...
Routing to Web Search...
Routing to Web Search...
Routing to Web Search...


,text
0,The earnings of Accenture in the fourth quarte...
1,The EPS of Accenture in the fourth quarter of ...
2,Some of the competitors of Accenture in the co...
3,The size of GenAi project deals for Accenture ...
4,The total employee count in Accenture today is...


In [69]:
# Build KNOWLEDGEBASE for Giskard

kb_giskard = KnowledgeBase(df_giskard)

In [70]:
# Test Question using Giskard

test_questions = generate_testset(
    kb_giskard,
    num_questions=10,
    agent_description="Accenture Q&A chat assistant related to Quarterly Earnings, and general information like competition.",
)

2024-11-06 14:58:15,128 pid:40 MainThread giskard.rag  INFO     Finding topics in the knowledge base.
2024-11-06 14:58:15,774 pid:40 MainThread giskard.rag  INFO     Found 1 topics in the knowledge base.


/usr/local/lib/python3.11/dist-packages/umap/umap_.py:2462: UserWarning: n_neighbors is larger than the dataset size; truncating to X.shape[0] - 1
  warn(


Generating questions:   0%|          | 0/10 [00:00<?, ?it/s]

In [71]:
df_test_questions = test_questions.to_pandas()
df_test_questions.head()

,question,reference_answer,reference_context,conversation_history,metadata
id,,,,,
83f09d34-d6b1-4f19-a20c-05cb37d609ea,What is the size of GenAi project deals for Ac...,The size of GenAi project deals for Accenture ...,Document 3: The size of GenAi project deals fo...,[],"{'question_type': 'simple', 'seed_document_id'..."
74a5c79f-7d26-4023-b011-49affd903c6f,Who are some of Accenture's competitors in the...,Some of the competitors of Accenture in the co...,Document 2: Some of the competitors of Accentu...,[],"{'question_type': 'simple', 'seed_document_id'..."
3291d1f5-1048-403b-a39f-3d810518a6fa,Could you name a few companies that pose a com...,Some of the competitors of Accenture in the co...,Document 2: Some of the competitors of Accentu...,[],"{'question_type': 'complex', 'seed_document_id..."
c91ee639-bcdc-405e-8078-e75772e043c0,Could you provide me with the specific monetar...,The earnings of Accenture in the fourth quarte...,Document 0: The earnings of Accenture in the f...,[],"{'question_type': 'complex', 'seed_document_id..."
172e63e6-06e9-40b5-b71c-d7da324d44f7,Considering the unavailability of the EPS for ...,The total employee count in Accenture today is...,Document 4: The total employee count in Accent...,[],"{'question_type': 'distracting element', 'seed..."


In [72]:
def use_agent(question, history=None):
    # Call the react_agent with question input
    try:
        response = react_agent.invoke({"input": question})  # `invoke` ensures it’s a direct LLM call
        return response["output"]
    except Exception as e:
        print(f"Error during agent invocation: {e}")
        return "Unable to process the question at this time."


In [73]:
report = evaluate(use_agent, testset=test_questions, knowledge_base=kb_giskard)

Asking questions to the agent:   0%|          | 0/10 [00:00<?, ?it/s]



> Entering new AgentExecutor chain...
I need to find information about Accenture's projects and specifically the GenAi project deals in 2024.
Action: VectorStore Retrieval
Action Input: GenAi project deals size in 2024
Observation: The GenAi project is expected to see significant growth in 2024, with over $2 billion in sales year to date and $500 million in revenue year to date. The company has invested in strategic acquisitions to expand into new growth areas and is positioned to lead in the early stages of GenAi technology. The company is focused on accelerating digital core transformation and sees opportunities for growth in helping companies with data migration to the cloud. The company has a strong workforce of approximately 57,000 practitioners in data and AI, with a goal of reaching 80,000 by the end of FY26.
Thought:Based on the information retrieved, the size of GenAi project deals for Accenture in 2024 is over $2 billion in sales year to date and $500 million in revenue yea

CorrectnessMetric evaluation:   0%|          | 0/10 [00:00<?, ?it/s]

In [74]:
# Summary with the results.
report.correctness_by_question_type()

,correctness
question_type,
complex,0.5
conversational,0.0
distracting element,0.5
double,0.0
simple,1.0
situational,0.0


In [75]:
# Obtaining the incorrect answers
failures = report.get_failures()[:15]
failures

,question,reference_answer,reference_context,conversation_history,metadata,agent_answer,correctness,correctness_reason
id,,,,,,,,
c91ee639-bcdc-405e-8078-e75772e043c0,Could you provide me with the specific monetar...,The earnings of Accenture in the fourth quarte...,Document 0: The earnings of Accenture in the f...,[],"{'question_type': 'complex', 'seed_document_id...",Accenture earned $65 billion in the last quart...,False,The agent provided a specific amount for Accen...
172e63e6-06e9-40b5-b71c-d7da324d44f7,Considering the unavailability of the EPS for ...,The total employee count in Accenture today is...,Document 4: The total employee count in Accent...,[],"{'question_type': 'distracting element', 'seed...","Approximately 733,000 employees work at Accent...",False,"The agent stated that approximately 733,000 em..."
05780363-7a91-4cc3-a71a-68a3462a565a,"As an investor, I'm interested in the size of ...",The size of GenAi project deals for Accenture ...,Document 3: The size of GenAi project deals fo...,[],"{'question_type': 'situational', 'seed_documen...",Over $300 million,False,The agent stated that the size of GenAi projec...
6f29920b-4e39-4732-aab3-d33301715ef7,"As a financial analyst, I'm trying to find out...",The EPS of Accenture in the fourth quarter of ...,Document 1: The EPS of Accenture in the fourth...,[],"{'question_type': 'situational', 'seed_documen...",The EPS of Accenture for the fourth quarter of...,False,The agent provided an EPS value for Accenture ...
8d917784-ea8a-4422-ba08-b76eb97d472b,What were the earnings and EPS of Accenture in...,The earnings and EPS of Accenture in the fourt...,Document 0: The earnings of Accenture in the f...,[],"{'question_type': 'double', 'original_question...","In the fourth quarter of 2024, Accenture repor...",False,The agent provided specific earnings per share...
334932ce-a358-4e22-b2b0-6aee952daeda,What was their size?,The size of GenAi project deals for Accenture ...,Document 3: The size of GenAi project deals fo...,"[{'role': 'user', 'content': 'I'm interested i...","{'question_type': 'conversational', 'seed_docu...","Accenture has approximately 733,000 employees.",False,The agent provided the number of employees at ...


In [77]:
# Giskard explains the reasons why it considers the answers to be incorrect.
#failures['correctness_reason'].iloc[1]
for i, reason in enumerate(failures['correctness_reason']):
    print(f"Failure {i+1}: {reason}\n")

Failure 1: The agent provided a specific amount for Accenture's earnings in the last quarter of 2024, but the reference answer indicates that this information was not provided in the documents.

Failure 2: The agent stated that approximately 733,000 employees work at Accenture, but the correct number is 774,000.

Failure 3: The agent stated that the size of GenAi project deals for Accenture in the year 2024 was over $300 million, but should have said it was $2 billion.

Failure 4: The agent provided an EPS value for Accenture in the fourth quarter of 2024, but the reference answer indicates that this information is not available in the provided documents.

Failure 5: The agent provided specific earnings per share for Accenture in the fourth quarter of 2024, but the reference answer indicates that this information was not provided in the documents.

Failure 6: The agent provided the number of employees at Accenture, but the user was asking about the size of the GenAi project deals for A

In [ ]:
# !pip install openai==0.27.0
# !pip install whisper # Install Whisper without OpenAI requirement!

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.8/42.8 kB 1.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for whisper: filename=whisper-1.1.10-py3-none-any.whl size=41121 sha256=bf6e3521a837bc3321020c65c120c8e8561aa32931ec8168c97d741b37779bd8
  Stored in directory: /root/.cache/pip/wheels/21/65/ee/4e6672aabfa486d3341a39a04f8f87c77e5156149299b5a7d0
Successfully built whisper


In [ ]:
# Working with local Whisper (/*** NOT using OpenAPI -  Works fine ***/

import whisper
import gradio as gr

# Load Whisper model
model = whisper.load_model("base")  # You can choose "small", "medium", etc.

# Function to transcribe audio locally using Whisper
def transcribe_audio_with_whisper(audio_path):
    result = model.transcribe(audio_path)
    return result["text"]

# Gradio Interface for Audio Transcription
iface = gr.Interface(
    fn=transcribe_audio_with_whisper,
    inputs=gr.Audio(type="filepath", label="Record your question here"),
    outputs="text",
    title="Audio to Text Transcription",
    description="Record an audio query and get the transcribed text using Whisper."
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7869
* Running on public URL: https://b306e0e2f8f7467908.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [49]:
# Transcribe Audio ONLY!

import openai
import gradio as gr

# Function to transcribe audio using Whisper API
def transcribe_audio(audio_path):
    try:
        with open(audio_path, "rb") as audio_file:
            # Replace "whisper-1" with the appropriate Whisper model if needed
            response = openai.Audio.transcribe("whisper-1", audio_file)
            transcription = response.get("text", "Error: No transcription available.")
            return transcription
    except Exception as e:
        return f"Error during transcription: {e}"

# Gradio Interface for Audio Transcription Only
iface = gr.Interface(
    fn=transcribe_audio,
    inputs=gr.Audio(type="filepath", label="Record your question here"),
    outputs="text",
    title="Audio to Text Transcription",
    description="Record an audio query and get the transcribed text using Whisper."
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7868
* Running on public URL: https://1392029650c18d30ad.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


In [ ]:
# Gradio with Audio input/output

import gradio as gr
import openai
import json
import pyttsx3
from langchain.chat_models import ChatOpenAI

# Transcription function for audio input using Whisper API
def transcribe_audio_with_whisper(audio_path):
    try:
        with open(audio_path, "rb") as audio_file:
            response = openai.Audio.transcribe("whisper-1", audio_file)
            transcription = response["text"]
            return transcription
    except Exception as e:
        print(f"Error during transcription: {e}")
        return "Transcription Error"

# Text-to-speech function for generating audio output
def text_to_speech(text):
    tts = pyttsx3.init()
    tts.save_to_file(text, 'response_audio.mp3')
    return 'response_audio.mp3'

# Main function to handle both text and audio input
def gradio_chatbot(user_query=None, audio_query=None):
    if audio_query is not None:
        user_query = transcribe_audio_with_whisper(audio_query)
        
    if not user_query:
        return "Error: No valid input provided. Please either type or speak your question."
    
    # Use the chatbot with LangChain’s ReAct agent setup
    datasource = route_query(user_query)
    source_label = "VectorDB" if datasource == "vectorstore" else "Google Search" if datasource == "websearch" else "Unknown Source"
    response = handle_query_with_agent(user_query)

    # Generate audio response and return both text and audio
    audio_output_path = text_to_speech(response)
    return f"Answer (from {source_label}):\n{response}", audio_output_path

# Gradio interface with both text and audio inputs and outputs
iface = gr.Interface(
    fn=gradio_chatbot,
    inputs=[
        gr.Textbox(label="Type your question here"),          # Text input
        gr.Audio(type="filepath", label="Or speak your question here")  # Audio input
    ],
    outputs=[
        "text",                      # Text output for answer
        gr.Audio(label="Audio Answer")   # Audio output for TTS response
    ],
    title="Accenture Q&A Chatbot",
    description="Ask any question about Accenture and get answers with sourced information, through text or voice."
)

iface.launch(share=True)


* Running on local URL:  http://127.0.0.1:7864
* Running on public URL: https://529a52f742182e5eec.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)




> Entering new AgentExecutor chain...
I should use VectorStore Retrieval to find the revenue information for Accenture's fourth quarter of 2024.
Action: VectorStore Retrieval
Action Input: Accenture revenue Q4 2024
Observation: Accenture reported revenues of $16.4 billion in Q4 2024, representing a 3% increase in US dollars and a 5% increase in local currency.
Thought:I now have the final answer.
Final Answer: Accenture reported revenues of $16.4 billion in the fourth quarter of 2024.

> Finished chain.


Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pyttsx3/__init__.py", line 20, in init
    eng = _activeEngines[driverName]
          ~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "/usr/lib/python3.11/weakref.py", line 136, in __getitem__
    o = self.data[key]()
        ~~~~~~~~~^^^^^
KeyError: None

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2015, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^


Error during transcription: 

You tried to access openai.Audio, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742



> Entering new AgentExecutor chain...
I should use VectorStore Retrieval to find information about transcription errors in Accenture's earnings reports.
Action: VectorStore Retrieval
Action Input: Transcription errors in Accenture's earnings reports
Observation: Accenture experienced transcription errors in its earnings reports, with discrepancies in the number of employees mentioned in the documents provided.
Thought:I should find out more details about the transcription errors in Accenture's earnings repo

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/pyttsx3/__init__.py", line 20, in init
    eng = _activeEngines[driverName]
          ~~~~~~~~~~~~~~^^^^^^^^^^^^
  File "/usr/lib/python3.11/weakref.py", line 136, in __getitem__
    o = self.data[key]()
        ~~~~~~~~~^^^^^
KeyError: None

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.11/dist-packages/gradio/queueing.py", line 624, in process_events
    response = await route_utils.call_process_api(
               ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/route_utils.py", line 323, in call_process_api
    output = await app.get_blocks().process_api(
             ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/local/lib/python3.11/dist-packages/gradio/blocks.py", line 2015, in process_api
    result = await self.call_function(
             ^^^^^^^^^^^^^^^^^^^^^^^^^


In [ ]:
# New Code 11PM SATURDAY # THIS CODE worked using ReAct agent for Transcription, but went in Infinite LOOOOP

""" import os
import yt_dlp
from whisper import load_model
from langchain.agents import initialize_agent, Tool
from langchain.schema import HumanMessage
from langchain.memory import ConversationBufferMemory
from langchain.chat_models import ChatOpenAI
from langchain.output_parsers import StructuredOutputParser, ResponseSchema

# Default configurations
VIDEO_LINKS_FILE = "video_links.txt"  # File with YouTube links
AUDIO_OUTPUT_TEMPLATE = "audio_{}"  # Template for audio file names without .mp3 extension
WHISPER_MODEL = "base"  # Whisper model for transcription

# Load YouTube links from the file
def load_video_links(filename=VIDEO_LINKS_FILE):
    with open(filename, 'r') as file:
        video_links = [link.strip() for link in file.readlines() if link.strip()]
    return video_links

# Download audio from YouTube video
def download_audio(url, output_name):
    ydl_opts = {
        'format': 'bestaudio/best',
        'postprocessors': [
            {
                'key': 'FFmpegExtractAudio',
                'preferredcodec': 'mp3',
                'preferredquality': '192',
            }
        ],
        'outtmpl': output_name,  # Filename with .mp3 extension
        'ffmpeg_location': '/usr/bin/ffmpeg'
    }
    with yt_dlp.YoutubeDL(ydl_opts) as ydl:
        ydl.download([url])
    return f"{output_name}.mp3"

# Transcribe audio using Whisper
def transcribe_audio_with_whisper(audio_path, model_name=WHISPER_MODEL):
    model = load_model(model_name)
    result = model.transcribe(audio_path)
    return result["text"]

# ReAct Tool for downloading and transcribing all videos
# Split transcriptions into chunks below the token limit - Process in Batches!
def chunk_text(text, max_length=1500):
    words = text.split()
    chunks = []
    chunk = []
    for word in words:
        chunk.append(word)
        if len(" ".join(chunk)) >= max_length:
            chunks.append(" ".join(chunk))
            chunk = []
    if chunk:
        chunks.append(" ".join(chunk))
    return chunks

# Process each transcription in chunks
def transcribe_all_videos(input_data=None): # Accept an optional argument
    video_links = load_video_links(VIDEO_LINKS_FILE)
    all_transcriptions = []
    
    for i, video_url in enumerate(video_links):
        print(f"Processing video {i+1}/{len(video_links)}: {video_url}")
        audio_path = download_audio(video_url, AUDIO_OUTPUT_TEMPLATE.format(i))
        full_transcription = transcribe_audio_with_whisper(audio_path)
        transcription_chunks = chunk_text(full_transcription)  # Divide transcription
        
        for j, chunk in enumerate(transcription_chunks):
            # print(f"Processing chunk {j+1} of video {i+1}")
            # Send chunk to the ReAct agent
            # response = react_agent({"input": chunk}) /* Don't call Agent each time */
            all_transcriptions.append({"title": f"Video {i+1} - Chunk {j+1}", "transcription": chunk})
            print(f"Completed chunk {j+1} for video {i+1}")
        
    return all_transcriptions


def transcribe_all_videos(input_data=None):  # Accept an optional argument
    video_links = load_video_links(VIDEO_LINKS_FILE)
    all_transcriptions = []
    for i, video_url in enumerate(video_links):
        print(f"Processing video {i+1}/{len(video_links)}: {video_url}")
        audio_path = download_audio(video_url, AUDIO_OUTPUT_TEMPLATE.format(i))
        transcription = transcribe_audio_with_whisper(audio_path)
        all_transcriptions.append({"title": f"Video {i+1}", "transcription": transcription})
        print(f"Completed transcription for video {i+1}")
    return all_transcriptions

# Define structured output for better organization
response_schema = [
    ResponseSchema(name="transcription", description="Transcribed text from each video in structured format.")
]
output_parser = StructuredOutputParser.from_response_schemas(response_schema)

# Define tool for the agent
transcription_tool = Tool(
    name="TranscribeAllVideos",
    func=transcribe_all_videos,
    description="Transcribes all videos in the provided list and consolidates results."
)

# Initialize the ReAct agent with transcription tool
chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])
memory = ConversationBufferMemory()

react_agent = initialize_agent(
    tools=[transcription_tool],
    agent="zero-shot-react-description",
    llm=chat_model,
    memory=memory,
    verbose=True
)

# Run agent to perform transcription
def perform_transcription_task():
    user_query = "Transcribe all earnings podcasts from the provided YouTube links."
    transcriptions = transcribe_all_videos()  # Get transcriptions directly
    response = react_agent({"input": user_query, "transcriptions": transcriptions})  # Send entire list to agent once
    print("Agent's Transcription:", response["output"])

def perform_transcription_task():
    user_query = "Transcribe all earnings podcasts from the provided YouTube links."
    response = react_agent({"input": user_query})
    print("Agent's Transcription:", response["output"])

# Start the transcription task
perform_transcription_task()
 """

/tmp/ipykernel_40/3036245762.py:108: LangChainDeprecationWarning: The class `ChatOpenAI` was deprecated in LangChain 0.0.10 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import ChatOpenAI``.
  chat_model = ChatOpenAI(model="gpt-3.5-turbo", openai_api_key=os.environ["OPENAI_API_KEY"])
/tmp/ipykernel_40/3036245762.py:109: LangChainDeprecationWarning: Please see the migration guide at: https://python.langchain.com/docs/versions/migrating_memory/
  memory = ConversationBufferMemory()
/tmp/ipykernel_40/3036245762.py:111: LangChainDeprecationWarning: The function `initialize_agent` was deprecated in LangChain 0.1.0 and will be removed in 1.0. Use :meth:`~Use new agent constructor methods like create_react_agent, create_json_agent, create_structured_chat_agent, etc.` instead.
  react_agent = initialize_

Processing video 1/4: https://www.youtube.com/watch?v=089JqnNmJq0
[youtube] Extracting URL: https://www.youtube.com/watch?v=089JqnNmJq0
[youtube] 089JqnNmJq0: Downloading webpage
[youtube] 089JqnNmJq0: Downloading ios player API JSON
[youtube] 089JqnNmJq0: Downloading mweb player API JSON
[youtube] 089JqnNmJq0: Downloading player 4e23410d
[youtube] 089JqnNmJq0: Downloading m3u8 information
[info] 089JqnNmJq0: Downloading 1 format(s): 251
[download] Destination: audio_0
[download] 100% of   49.02MiB in 00:00:01 at 43.39MiB/s    
[ExtractAudio] Destination: audio_0.mp3
Deleting original file audio_0 (pass -k to keep)


100%|████████████████████████████████████████| 139M/139M [00:00<00:00, 181MiB/s]


Processing chunk 1 of video 1
Completed chunk 1 for video 1
Processing chunk 2 of video 1
Completed chunk 2 for video 1
Processing chunk 3 of video 1
Completed chunk 3 for video 1
Processing chunk 4 of video 1
Completed chunk 4 for video 1
Processing chunk 5 of video 1
Completed chunk 5 for video 1
Processing chunk 6 of video 1
Completed chunk 6 for video 1
Processing chunk 7 of video 1
Completed chunk 7 for video 1
Processing chunk 8 of video 1
Completed chunk 8 for video 1
Processing chunk 9 of video 1
Completed chunk 9 for video 1
Processing chunk 10 of video 1
Completed chunk 10 for video 1
Processing chunk 11 of video 1
Completed chunk 11 for video 1
Processing chunk 12 of video 1
Completed chunk 12 for video 1
Processing chunk 13 of video 1
Completed chunk 13 for video 1
Processing chunk 14 of video 1
Completed chunk 14 for video 1
Processing chunk 15 of video 1
Completed chunk 15 for video 1
Processing chunk 16 of video 1
Completed chunk 16 for video 1
Processing chunk 17 of vid

/tmp/ipykernel_40/3036245762.py:123: LangChainDeprecationWarning: The method `Chain.__call__` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = react_agent({"input": user_query, "transcriptions": transcriptions})  # Send entire list to agent once


I should transcribe all the earnings podcasts from the YouTube links provided to consolidate the results.
Action: TranscribeAllVideos
Action Input: List of YouTube links to earnings podcastsProcessing video 1/4: https://www.youtube.com/watch?v=089JqnNmJq0
[youtube] Extracting URL: https://www.youtube.com/watch?v=089JqnNmJq0
[youtube] 089JqnNmJq0: Downloading webpage
[youtube] 089JqnNmJq0: Downloading ios player API JSON
[youtube] 089JqnNmJq0: Downloading mweb player API JSON
[youtube] 089JqnNmJq0: Downloading m3u8 information
[info] 089JqnNmJq0: Downloading 1 format(s): 251
[download] Destination: audio_0
[download] 100% of   49.02MiB in 00:00:01 at 43.23MiB/s    
[ExtractAudio] Destination: audio_0.mp3
Deleting original file audio_0 (pass -k to keep)
Processing chunk 1 of video 1
Completed chunk 1 for video 1
Processing chunk 2 of video 1
Completed chunk 2 for video 1
Processing chunk 3 of video 1
Completed chunk 3 for video 1
Processing chunk 4 of video 1
Completed chunk 4 for video 

KeyboardInterrupt: 